<a href="https://colab.research.google.com/github/rohanreddych/symmetrical-guacamole/blob/master/ad-success/Ad_Success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
#import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train = pd.read_csv("gdrive/My Drive/Dataset/Train.csv")
test = pd.read_csv("gdrive/My Drive/Dataset/Test.csv")

In [0]:
s = (train.dtypes =="object")
categorical_columns = list(s[s].index)

In [0]:
#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for c in categorical_columns:
    train[c] = le.fit_transform(train[c])
    test[c] = le.fit_transform(test[c])
    

In [0]:
#minmax scaler
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
train["average_runtime(minutes_per_week)"] = mm.fit_transform(train[["average_runtime(minutes_per_week)"]])
test["average_runtime(minutes_per_week)"] = mm.fit_transform(test[["average_runtime(minutes_per_week)"]])

In [0]:
#test train split
test_ids = test["id"]
test = test.drop(["id"], axis = 1)
X = train.drop(["id", "netgain"], axis=1)
y = train["netgain"].astype(int)
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size = 0.25)


In [0]:
#defining the model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
cv = StratifiedShuffleSplit(n_splits=10, test_size=.20)
C_vals = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,16.5,17,17.5,18]
penalties = ["l1", "l2"]
params = {'penalty':penalties, "C":C_vals}
grid = GridSearchCV(estimator=LogisticRegression(solver="liblinear"), n_jobs=-1, scoring="accuracy", param_grid=params)

In [9]:
grid.fit(train_X, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3,
                               4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 16.5,
                               17, 17.5, 18],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
   

In [0]:
logreg_grid = grid.best_estimator_

In [11]:
logreg_grid.score(X,y)

0.796875

    For Different Algorithms
   

In [0]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
k_range = (1,31)
weights_options = ["uniform", "distance"]
cv = StratifiedShuffleSplit(n_splits=15, random_state=96, test_size=0.3)
params = {"n_neighbors":k_range, "weights":weights_options}
grid_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=params, cv=cv, n_jobs=-1)

In [13]:
grid_knn.fit(X,y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=15, random_state=96, test_size=0.3,
            train_size=None),
             error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'n_neighbors': (1, 31),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
grid_knn.best_estimator_.score(X,y)
knn_grid = grid_knn.best_estimator_

In [15]:
#GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gaussian = GaussianNB()
gaussian.fit(X,y)
y_pred_gn = gaussian.predict(val_X)
print(accuracy_score(y_pred_gn, val_y))

0.8072788697788698


In [0]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
max_depth = range(1, 31)
max_feature = ["auto"]
criterion = ["entropy", "gini"]
param = {"max_depth":max_depth, "max_features":["auto"], "criterion":criterion}

dec_grid = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param, cv = StratifiedShuffleSplit(n_splits=10, random_state=42), n_jobs=-1)


In [19]:
dec_grid.fit(X,y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=None,
            train_size=None),
             error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'cri

In [70]:
dec_tree_grid = dec_grid.best_estimator_
dec_tree_grid.score(X,y)

0.8197558353808354

In [73]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
n_estimators = [70,80,150,160, 170,175,180,185];
max_depth = range(1,10);
criterions = ['gini', 'entropy'];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)


parameters = {'n_estimators':n_estimators,
              'max_depth':max_depth,
              'criterion': criterions
              
        }
grid = GridSearchCV(estimator=RandomForestClassifier(max_features='auto'),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid.fit(X,y) 
rf_grid = grid.best_estimator_
rf_grid.score(X,y)

0.8229422604422605

In [42]:
#Bagging
from sklearn.ensemble import BaggingClassifier
#n_estimators = [10,30,50,70,80,150,160, 170,175,180,185];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)

parameters = {'n_estimators':[10, 30, 50, 70, 90]}
grid = GridSearchCV(BaggingClassifier(base_estimator= None, ## If None, then the base estimator is a decision tree.
                                      bootstrap_features=False),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid.fit(X,y) 
bagging_grid = grid.best_estimator_
bagging_grid.score(X,y)

0.8591062653562653

In [44]:
#Adaboost
from sklearn.ensemble import AdaBoostClassifier
n_estimators = [100,140,145,150,160, 170,175,180,185];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
learning_r = [0.1,1,0.01,0.5]

parameters = {'n_estimators':n_estimators,
              'learning_rate':learning_r
              
        }
grid = GridSearchCV(AdaBoostClassifier(base_estimator= None, ## If None, then the base estimator is a decision tree.
                                     ),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid.fit(X,y) 
adaBoost_grid = grid.best_estimator_
adaBoost_grid.score(X,y)

0.8184121621621622

In [52]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

gradient_boost = GradientBoostingClassifier()
gradient_boost.fit(train_X, train_y)
y_pred = gradient_boost.predict(val_X)
gradient_accy = round(accuracy_score(y_pred, val_y), 3)
print(gradient_accy)

0.819


In [53]:
#Extra tree classifier
from sklearn.ensemble import ExtraTreesClassifier
ExtraTreesClassifier = ExtraTreesClassifier()
ExtraTreesClassifier.fit(X, y)
y_pred = ExtraTreesClassifier.predict(val_X)
extraTree_accy = round(accuracy_score(y_pred, val_y), 3)
print(extraTree_accy)

0.854


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
#XGboost
from xgboost import XGBRFClassifier
from xgboost import XGBClassifier
pars = {"max_depth":range(1, 10), "n_estimators":[50, 70, 100], "learning_rate":[0.01, 0.1, 0.5, 1], }
xgb_grid = GridSearchCV(estimator=XGBClassifier(booster="gbtree", random_state=42), param_grid = pars, n_jobs=-1, cv=StratifiedShuffleSplit(n_splits=11, test_size=0.2))
xgbrf_grd = GridSearchCV(estimator=XGBRFClassifier(random_state=69), param_grid=pars, n_jobs=-1)

In [118]:
xgb_grid.fit(X,y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=11, random_state=None, test_size=0.2,
            train_size=None),
             error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100,...,
                                     nthread=None, objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5, 1],
 

In [119]:
y_pred = xgb_grid.predict(val_X)
XGBC = xgb_grid.best_estimator_
print(accuracy_score(y_pred, val_y))

0.8157248157248157


In [113]:
xgbrf_grd.fit(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRFClassifier(base_score=0.5, colsample_bylevel=1,
                                       colsample_bynode=0.8, colsample_bytree=1,
                                       gamma=0, learning_rate=1,
                                       max_delta_step=0, max_depth=3,
                                       min_child_weight=1, missing=None,
                                       n_estimators=100, n_jobs=1, nthread=None,
                                       objective='binary:logistic',
                                       random_state=69, reg_alpha=0,
                                       reg_lambda=1, scale_pos_weight=1,
                                       seed=None, silent=None, subsample=0.8,
                                       verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5, 1],
                         'max_depth': range(1, 10),
     

In [78]:
#Voting Finally
from sklearn.ensemble import VotingClassifier

voting_classifier = VotingClassifier(estimators=[
    ('lr_grid', logreg_grid),
    #('svc', svm_grid),
    ('random_forest', rf_grid),
    ('gradient_boosting', gradient_boost),
    ('decision_tree_grid',dec_tree_grid),
    ('knn_classifier', knn_grid),
    ('XGB_Classifier', XGBC),
    ('bagging_classifier', bagging_grid),
    ('adaBoost_classifier',adaBoost_grid),
    ('ExtraTrees_Classifier', ExtraTreesClassifier),
    ('gaussian_classifier',gaussian),
    #('gaussian_process_classifier', GaussianProcessClassifier)
],voting='hard')


voting_classifier = voting_classifier.fit(X,y)

y_pred = voting_classifier.predict(val_X)
voting_accy = accuracy_score(y_pred, val_y)
print(voting_accy)

0.8278562653562653


In [0]:
sub_pred1 = xgbrf_grd.best_estimator_.predict(test)

In [0]:
subm = pd.DataFrame({"id":test_ids, "netgain":sub_pred1.astype(bool)})
subm.to_csv("x2_sub.csv", index=False)

In [10]:
from tensorflow import feature_column
import tensorflow as tf 
from tensorflow.keras import layers
train.head()

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,19717,3,0,0,1,0.448980,2,39,0.027465,0,0,False
1,31593,2,4,0,1,0.448980,2,39,0.027465,1,0,False
2,5681,0,2,0,0,0.448980,2,39,0.027465,0,1,False
3,15491,5,5,3,0,0.397959,2,39,0.027465,1,0,False
4,23587,2,4,0,1,0.479592,2,39,0.027465,0,0,True


In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop("netgain")
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(df))
    ds = ds.batch(batch_size)
    return ds

In [0]:
train_ds = df_to_dataset(train_X)

In [0]:
sample = train_ds.take(1)


Switching to new notebook for other algos
